In [15]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv('combined_samples/combined_data.csv')

In [3]:
# Eliminate first unnecessary row.
df = df.iloc[:, 1:]

In [4]:
df.head()

,timestamp,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,...,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
0,2013-03-27 04:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-77.0,...,0.0,599.0,0.0,0.0,0.0,274.0,0.0,185.0,140.0,NaN
1,2013-03-27 04:10:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,600.0,0.0,0.0,0.0,557.0,1.0,42.0,0.0,NaN
2,2013-03-27 04:20:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-71.5,...,0.0,600.0,0.0,0.0,0.0,457.0,0.0,143.0,0.0,NaN
3,2013-03-27 04:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,208.0,600.0,0.0,0.0,0.0,564.0,1.0,35.0,0.0,NaN
4,2013-03-27 04:40:00,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-62.0,...,600.0,600.0,0.0,0.0,0.0,127.0,94.0,379.0,0.0,NaN


In [5]:
def create_same_length_instances(df, length=72, label='STRESSED'):
    df = df.drop(columns=['timestamp'])
    full_data = pd.DataFrame()
    indexes = list(df[df[label].notnull()].index)
    start = 0
    for i in indexes:
        if i - start >= length:
            instance = df.iloc[i-length+1:i+1, :]
            full_data = full_data.append(instance, ignore_index=True, sort=False)
        else:
            index_diff = i - start
            instance = df.iloc[i-index_diff+1:i+1, :]
            back_fill = np.empty((length-index_diff, df.shape[1]))
            back_fill.fill(np.nan)
            back_fill = pd.DataFrame(back_fill, columns=df.columns)
            instance = back_fill.append(instance, ignore_index=True, sort=False)
            full_data = full_data.append(instance, ignore_index=True, sort=False)
        start = i
    return full_data

In [6]:
uza = create_same_length_instances(df, length=72)

In [7]:
np.array([np.random.rand(5,3), 1])

array([array([[0.55882077, 0.52179584, 0.29380415],
       [0.10089345, 0.83290993, 0.62658299],
       [0.66704723, 0.43893695, 0.63632733],
       [0.32894024, 0.26739966, 0.61307538],
       [0.10952601, 0.27900987, 0.96245161]]),
       1], dtype=object)

In [106]:
def create_numpy_array(df, label='STRESSED'):
    indexes = list(df[df[label].notnull()].index)
    labels = df.loc[indexes, 'STRESSED'].values
    data = df.drop(columns=['STRESSED'])
    all_data = np.empty((np.empty((indexes[1] - indexes[0], data.shape[1])), len(indexes)))
    start = 0
    j = 0
    for i in indexes:
        instance = np.array(data.iloc[start:i+1, :].values, labels[j])
        if j == 0:
            all_data[j] = instance
        else:
            all_data.append(instance, axis=0)
        start = i+1
        j += 1
    return all_data, labels

In [107]:
create_numpy_array(uza)

TypeError: only integer scalar arrays can be converted to a scalar index

In [8]:
uza[uza.STRESSED.notnull()].shape

(2347, 26)

In [9]:
uza[uza.STRESSED == 0].shape

(733, 26)

In [10]:
uza[uza.STRESSED == 1].shape

(1614, 26)

In [18]:
def create_instances(df, label='STRESSED'):
    indexes = list(df[df[label].notnull()].index)
    data = df.drop(columns=[label])
    start = 0
    all_data = []
    for i in indexes:
        all_data.append([torch.from_numpy(data.iloc[j, :].values) for j in range(start, i+1)])
        start = i+1
    return all_data

In [20]:
a = create_instances(uza)

In [25]:
len(a[2])

72

In [11]:
uza[uza.STRESSED.notnull()]

,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,wifi_level_std,...,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
71,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-61.000000,NaN,...,0.0,600.0,0.0,0.0,0.0,595.0,2.0,3.0,0.0,1.0
143,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,600.0,0.0,0.0,0.0,356.0,2.0,242.0,0.0,1.0
215,0.0,-89.0,5.618846,7.0,3.0,3.0,1.0,0.0,-84.000000,8.447316,...,600.0,598.0,2.0,0.0,0.0,600.0,0.0,0.0,0.0,1.0
287,569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,600.0,600.0,0.0,0.0,0.0,49.0,510.0,41.0,0.0,1.0
359,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.000000,20.126268,...,0.0,600.0,0.0,0.0,0.0,40.0,536.0,24.0,0.0,0.0
431,600.0,-80.0,11.095545,10.0,6.0,2.0,0.0,2.0,-72.000000,24.005050,...,0.0,600.0,0.0,0.0,0.0,224.0,356.0,20.0,0.0,0.0
503,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,592.0,0.0,0.0,8.0,385.0,198.0,17.0,0.0,0.0
575,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.000000,25.974919,...,600.0,600.0,0.0,0.0,0.0,89.0,325.0,186.0,0.0,1.0
647,0.0,-88.0,11.789826,4.0,0.0,3.0,0.0,1.0,-85.000000,8.417240,...,600.0,600.0,0.0,0.0,0.0,127.0,382.0,91.0,0.0,1.0
719,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-81.875000,10.645335,...,0.0,600.0,0.0,0.0,0.0,0.0,159.0,441.0,0.0,0.0
